In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
BASE = "/home/usertp/FDT/sf_FDT/CorpusARIA/TERMYT"

In [23]:
rows = []

for event in sorted(os.listdir(BASE)):
    raw_dir = os.path.join(BASE, event, "default-ann", "raw")
    term_file = os.path.join(raw_dir, "termList.txt")

    if os.path.exists(term_file):

        # Lecture brute du fichier à colonnes multiples
        df = pd.read_csv(
            term_file,
            sep="\t",
            header=None,
            engine="python",
            quoting=3
        )

        # Colonne 1 = vrai terme (colonne 0 = ID inutile)
        df["term"] = df[1].astype(str)

        # Filtrer "Head", "nan", etc.
        df = df[df["term"].str.lower() != "head"]
        df = df[df["term"].str.lower() != "nan"]

        # ❗ IMPORTANT : filtrer tous les termes numériques
        df = df[~df["term"].str.match(r"^\d+$")]             # ex: "40"
        df = df[df["term"].str.contains("[A-Za-zÀ-ÿ]")]      # garder termes linguistiques

        # Ajouter l'évènement
        df["event"] = event

        # On ne garde que la structure minimale
        rows.append(df[["event", "term"]])

# Fusion finale
df_all = pd.concat(rows, ignore_index=True)

df_all.head()

,event,term
0,event-0,Lemmatised form
1,event-0,orientation du pale
2,event-0,bloc de frein du système
3,event-0,bloc de frein
4,event-0,limite de conception de le éolienne


In [24]:
df_all["term"].unique()[:50]

array(['Lemmatised form', 'orientation du pale',
       'bloc de frein du système', 'bloc de frein',
       'limite de conception de le éolienne',
       'contrôle annuels.Les autre éolienne du site',
       'protocole du inspection de maintenance',
       'système de contrôle du pale', 'expertise de le éolien mise',
       'partie du bloc de frein', 'autre éolienne du parc',
       'investigation de le exploitant', 'pale à le consigne',
       'éolienne de 60 m', 'vitesse du rotor', 'remplacement de tout',
       'limite de sécurité', 'interprétation du donnée',
       'début de matinée', 'défaillance du système',
       'dispositif de protection', 'condition météorologique',
       'augmentation rapide', 'position effectif', 'charge mécanique',
       'intervention direct', 'liaison mécanique',
       'défaillance potentiel', 'm supérieur', 'usure anormal',
       'éolienne', 'pale', 'système', 'orientation', 'bloc', 'frein',
       'sécurité', 'place', 'défaillance', 'rotor', 'autre

In [30]:
df_all["term"] = df_all["term"].astype(str)
df_all.head()

,event,term
0,event-0,Lemmatised form
1,event-0,orientation du pale
2,event-0,bloc de frein du système
3,event-0,bloc de frein
4,event-0,limite de conception de le éolienne


In [31]:
# Regrouper les termes par événement
event_terms = df_all.groupby("event")["term"].apply(list)

# Affichage rapide
event_terms.head()

event
event-0      [Lemmatised form, orientation du pale, bloc de...
event-1      [Lemmatised form, défaut lier au détecteur de ...
event-10     [Lemmatised form, origine de le erreur de pitc...
event-100    [Lemmatised form, parc éolien, suivi de le mor...
event-101    [Lemmatised form, bout de pale, autre aérogéné...
Name: term, dtype: object

In [32]:
# Un document = tous les termes concaténés
corpus = {
    event: " ".join(terms)
    for event, terms in event_terms.items()
}

len(corpus), list(corpus.items())[:3]

(285,
 [('event-0',
   'Lemmatised form orientation du pale bloc de frein du système bloc de frein limite de conception de le éolienne contrôle annuels.Les autre éolienne du site protocole du inspection de maintenance système de contrôle du pale expertise de le éolien mise partie du bloc de frein autre éolienne du parc investigation de le exploitant pale à le consigne éolienne de 60 m vitesse du rotor remplacement de tout limite de sécurité interprétation du donnée début de matinée défaillance du système dispositif de protection condition météorologique augmentation rapide position effectif charge mécanique intervention direct liaison mécanique défaillance potentiel m supérieur usure anormal éolienne pale système orientation bloc frein sécurité place défaillance rotor autre m éolienne intervention mât exploitant maintenance usure position tempête sol remplacement contrôle conception 3 pale de le éolienne survitesse protection contrôle parc diagnostic contrôle annuels.Les autre même typ

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words=None,
    token_pattern=r"(?u)\b\w[\w\-']+\b"
)

X = vectorizer.fit_transform(corpus.values())
events = list(corpus.keys())

X.shape

(285, 2222)

In [36]:
categories = {
    "environmental": ["oiseau", "animal", "impact", "pollution", "habitat", "bruit", "sol", "eau"],
    "economic": ["coût", "production", "arrêt", "dégât", "réparation", "maintenance", "perte"],
    "social": ["blessure", "accident", "pompiers", "sécurité", "personnel"]
}


In [37]:
def classify_event(terms):
    scores = {cat: 0 for cat in categories}
    
    for t in terms:
        if not isinstance(t, str):
            continue
        
        t_low = t.lower()
        
        for cat, kw in categories.items():
            if any(k in t_low for k in kw):
                scores[cat] += 1
    
    best = max(scores, key=scores.get)
    return best if scores[best] > 0 else "non-classe"


In [38]:
event_classes = {
    event: classify_event(terms)
    for event, terms in event_terms.items()
}

event_classes


{'event-0': 'social',
 'event-1': 'environmental',
 'event-10': 'economic',
 'event-100': 'environmental',
 'event-101': 'environmental',
 'event-102': 'economic',
 'event-103': 'environmental',
 'event-104': 'economic',
 'event-105': 'economic',
 'event-106': 'economic',
 'event-107': 'environmental',
 'event-108': 'environmental',
 'event-109': 'environmental',
 'event-11': 'environmental',
 'event-110': 'environmental',
 'event-111': 'economic',
 'event-112': 'environmental',
 'event-113': 'environmental',
 'event-114': 'environmental',
 'event-115': 'environmental',
 'event-116': 'environmental',
 'event-117': 'environmental',
 'event-118': 'economic',
 'event-119': 'economic',
 'event-12': 'economic',
 'event-120': 'economic',
 'event-121': 'environmental',
 'event-122': 'environmental',
 'event-123': 'economic',
 'event-124': 'environmental',
 'event-125': 'environmental',
 'event-126': 'environmental',
 'event-127': 'environmental',
 'event-128': 'social',
 'event-129': 'environ

In [39]:
pd.DataFrame.from_dict(event_classes, orient="index", columns=["class"]).head(10)


,class
event-0,social
event-1,environmental
event-10,economic
event-100,environmental
event-101,environmental
event-102,economic
event-103,environmental
event-104,economic
event-105,economic
event-106,economic


In [43]:
topn = 20
feature_names = vectorizer.get_feature_names()
tfidf_means = X.mean(axis=0).A1  # moyenne TF-IDF sur tout le corpus

top_ids = tfidf_means.argsort()[::-1][:topn]
top_terms = [feature_names[i] for i in top_ids]

top_terms

['de',
 'du',
 'le',
 'parc',
 'éolien',
 'mortalité',
 'éolienne',
 'suivi',
 'pale',
 'aviaire',
 'système',
 'sol',
 'espèce',
 'cadavre',
 'chute',
 'milan',
 'autre',
 'collision',
 'au',
 'royal']